In [1]:
import nltk, pickle, string
from nltk.corpus import stopwords
import re
import os
from tika import parser
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


from textblob.classifiers import NaiveBayesClassifier as NBC
from textblob import TextBlob


Using TensorFlow backend.


In [21]:
def clean_doc(doc):
    """
    Cleaning a document by several methods:
        - Lowercase
        - Removing whitespaces
        - Removing stopwords
        - Removing punctuations
        
    """
    stop_words = set(stopwords.words('english'))
    
    # Lowercase
    doc = doc.lower()
   
    tokens = doc.translate({ord(c):"" for c in "\u200b\uf020\u2028\xa0\uf0e0\uf095\uf041\uf0e1\uf0b7\xad"})
    tokens = tokens.translate({ord(c):" " for c in "[):,·](;•●■♦▪"})
    tokens = tokens.translate({ord(c):"f" for c in "�"})
    
    # Removing multiple whitespaces
    tokens = re.sub(r"\?", " \? ", tokens)
    
    # Split in tokens
    tokens = tokens.split()
    
    # Remove Stopwords
    tokens = [w for w in tokens if not w in stop_words]
    # Remove punctuation
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens]
    
    return ' '.join(tokens)

In [22]:
def LoadData_others():
    label_others=[]
    i=0
    for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level4/tech/Others/others"):
        try:
            print (i, file)
            parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level4/tech/Others/others/"+file)

            tech_contents=clean_doc(parsedPDF['content'])
            #Data = resume_contents.encode('utf-8')    
            label_others.append((tech_contents,'others'))
        except UnicodeEncodeError:
            print ('Unicode error:', file)
        i=i+1
    #print (label_resume)
    return(label_others)
        

In [23]:
def LoadData_webdesigner():
    label_webdesigner=[]
    i=1
    for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level4/tech/Others/webdesigner"):
        try:
            print (i, file)
            parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level4/tech/Others/webdesigner/"+file)
        
            it_contents=clean_doc(parsedPDF['content'])
       
            
            label_webdesigner.append((it_contents, 'webdesigner'))
        except UnicodeEncodeError:
            print ('Unicode error:', file)
        i=i+1
            
    return(label_webdesigner)

In [24]:
label_others=LoadData_others()

0 RajibDebnathProfile.pdf
1 NeriinnGopuProfile(1).pdf
2 NataliaMolchanovaProfile.pdf
3 SimonaTedescoProfile.pdf
4 3d Character Animator Resume.doc
5 SagaciousDesignProfile.pdf
6 LaurenHamillProfile.pdf
7 CostantinoMontanariProfile.pdf
8 IlariaAlquatiProfile(1).pdf
9 Martina GinevraAlbanoProfile.pdf
10 LorenzoDe AngelisProfile.pdf
11 AtaHaghdarProfile.pdf
12 LorenzoDe AngelisProfile(2).pdf
13 ValerioBailo ModestiProfile.pdf
14 SoniaDonizProfile.pdf
15 StevaDokmanacProfile.pdf
16 KimberlyPelkeyProfile.pdf
17 SimonaMuscogiuriProfile.pdf
18 TureNicotraProfile.pdf
19 Ramana RaoPanduriProfile.pdf
20 ImmaPezzellaProfile.pdf
21 ZenithJ JosephProfile.pdf
22 AlexFreiProfile.pdf
23 MariaMarangoloProfile.pdf
24 REJIDHESIProfile.pdf
25 KiranMohodProfile.pdf
26 NicolaCredentinoProfile.pdf
27 SimoneAlbieriProfile.pdf
28 SaraRiscicaProfile.pdf
29 Suri.Profile.pdf
30 GianlucaBaldacciniProfile.pdf
31 Creative Resume.pdf
32 RobertaAnnicchiaricoProfile.pdf
33 IlariaAlquatiProfile.pdf
34 LaviniaBizzarriPro

In [25]:
len(label_others)

150

In [26]:
data_labels=[]
data=[]

for row in label_others:
    data.append(row[0])
    data_labels.append(row[1])

In [27]:
label_webdesigner=LoadData_webdesigner()

1 nilima_balkhande_Pune_1.07_yrs.doc
2 RAHUL_KUMAR_Delhi_4.08_yrs.doc
3 ankita_tailor_India_5.00_yrs.docx
4 Aravind Syam Prakash.doc
5 jasmendar_kumar_Delhi_3.00_yrs.doc
6 Saumya__Delhi_3.00_yrs.docx
7 Nagaraju_S_Mysore_4.05_yrs.doc
8 GOBI.pdf
9 Govind_Narayanan_Cochin___Kochi___Ernakulam_10.00_yrs.doc
10 Ankit_Kumar_Bengaluru___Bangalore_1.02_yrs.docx
11 Kiran.pdf
12 Anitha_P_Bengaluru___Bangalore_0.00_yrs.docx
13 naga_chaitanya_Hyderabad___Secunderabad_3.08_yrs.doc
14 Hasan_Ali_Alavudeen_Madurai_9.00_yrs.docx
15 Kalpesh_gurav_Thane_1.00_yrs.docx
16 FREDIN FRANCIS.doc
17 Raju_Jinukala_Hyderabad___Secunderabad_2.08_yrs.doc
18 Mohd_Imran_Siddiqui_Delhi_4.00_yrs.doc
19 mamatha_gogi_Hyderabad___Secunderabad_6.00_yrs.doc
20 rafia_khan_Delhi_1.00_yrs.docx
21 Harikrishnan S - Cochin _ Kochi _ Ernakulam, 3 years.pdf
22 Anand_Shendage_Mumbai_1.06_yrs.pdf
23 Nihal_Gesudraz_Bengaluru___Bangalore_4.02_yrs.pdf
24 Chandrajit_Das_Hyderabad___Secunderabad_0.00_yrs.docx
25 Neeta_Gotad_Mumbai_1.05_yrs.

In [28]:
for row in label_webdesigner:
    data.append(row[0])
    data_labels.append(row[1])

In [29]:
print(len(data))
print(len(data_labels))

300
300


In [30]:
print(len(data_labels))

300


In [31]:
#labels = [0 for _ in range(len(label_resume))] + [1 for _ in range(len(label_jd))]
#print(labels)


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=4, max_df=0.9)

In [33]:
# Apply model on jdres train data
train_vectors = vectorizer.fit_transform(data)
print(train_vectors)

  (0, 1616)	0.0685349592197
  (0, 1047)	0.327583370131
  (0, 710)	0.172911697889
  (0, 229)	0.102936061445
  (0, 265)	0.148378694678
  (0, 183)	0.208149771303
  (0, 2153)	0.0212553459342
  (0, 1152)	0.054557893038
  (0, 96)	0.0769117940942
  (0, 2424)	0.0608261538177
  (0, 1581)	0.0718352254972
  (0, 1494)	0.0479899750652
  (0, 1049)	0.0814664750847
  (0, 733)	0.0755111028625
  (0, 541)	0.0721074695095
  (0, 1079)	0.181009015249
  (0, 1177)	0.0394204420843
  (0, 648)	0.0459199353141
  (0, 336)	0.0379824373603
  (0, 1081)	0.0769117940942
  (0, 1374)	0.116895846463
  (0, 861)	0.0741609939121
  (0, 542)	0.0680434864004
  (0, 706)	0.187581253339
  (0, 1615)	0.0532450975799
  :	:
  (299, 2248)	0.0878765158225
  (299, 2194)	0.0770400354269
  (299, 1660)	0.0878765158225
  (299, 2102)	0.0911360559524
  (299, 1257)	0.244643842471
  (299, 681)	0.0437014639602
  (299, 1091)	0.0432048216235
  (299, 682)	0.0415677856793
  (299, 991)	0.0585168103681
  (299, 904)	0.0461247427998
  (299, 387)	0.119418

In [34]:

X_train, X_test, y_train, y_test = train_test_split(train_vectors, data_labels, test_size=0.2, random_state=42)    

In [35]:
svm_model = svm.SVC(kernel='linear') 
svm_model.fit(X_train, y_train) 
svm_prediction = svm_model.predict(X_train)
import numpy as np
print ('svm:', np.mean(svm_prediction == y_train))

'''for i in range(80):
    print ('>>>>>>>>>>>.',model.predict(train_vectors[i]))'''

    
from sklearn.ensemble import RandomForestClassifier  
RF_model = RandomForestClassifier(max_depth=2, random_state=0)
RF_model.fit(X_train, y_train) 
RF_prediction = RF_model.predict(X_train)
import numpy as np
print ('Random Forest:', np.mean(RF_prediction == y_train))

from sklearn.linear_model import SGDClassifier
SGD_model = SGDClassifier()
SGD_model.fit(X_train, y_train) 
SGD_prediction = SGD_model.predict(X_train)
import numpy as np
print ('SGD:', np.mean(SGD_prediction == y_train))

from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train,y_train)
dtree_predictions = dtree_model.predict(X_train)
print ('DT:', np.mean(dtree_predictions == y_train))


from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train,y_train)
knn_predictions = knn.predict(X_train) 
print ('knn:', np.mean(knn_predictions == y_train))



svm: 1.0
Random Forest: 0.979166666667
SGD: 1.0
DT: 0.970833333333
knn: 0.9875


/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [36]:
for i in range(240):
    print ('>>>>>>>>>>>.',RF_model.predict(X_train[i]))
    print (y_train[i])
print(len(y_train))  

>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['ot

In [37]:
svm_model_test = svm.SVC(kernel='linear') 
svm_model_test.fit(X_test, y_test) 
svm_prediction_test = svm_model_test.predict(X_test)
print ('svm_test:', np.mean(svm_prediction_test == y_test))

svm_test: 1.0


In [38]:
from sklearn.ensemble import RandomForestClassifier  
RF_model_test = RandomForestClassifier(max_depth=2, random_state=0)
RF_model_test.fit(X_test, y_test) 
RF_prediction_test = RF_model_test.predict(X_test)
print ('RandomForest_test:', np.mean(RF_prediction_test == y_test))

RandomForest_test: 0.966666666667


In [39]:
from sklearn.linear_model import SGDClassifier
SGD_model_test = SGDClassifier()
SGD_model_test.fit(X_test, y_test) 
SGD_prediction_test = SGD_model_test.predict(X_test)
print ('SGD_test:', np.mean(SGD_prediction_test == y_test))


SGD_test: 1.0


/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [40]:
from sklearn.tree import DecisionTreeClassifier
dtree_model_test = DecisionTreeClassifier(max_depth = 2).fit(X_test, y_test)
dtree_predictions_test = dtree_model_test.predict(X_test)
print ('DT_test:', np.mean(dtree_predictions_test == y_test))

from sklearn.neighbors import KNeighborsClassifier
knn_test = KNeighborsClassifier(n_neighbors = 7).fit(X_test, y_test)
knn_predictions_test = knn_test.predict(X_test) 
print ('knn_test:', np.mean(knn_predictions_test == y_test))


DT_test: 1.0
knn_test: 0.95


In [42]:
"""for i in range(20):
    print ('>>>>>>>>>>>.',RF_model.predict(jdres_test_vectors[i]))
    print (jdres_test_labels[i])"""

for i in range(300):
    print ('>>>>>>>>>>>.',svm_model.predict(train_vectors[i]))
    print (data_labels[i])


>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>

>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['w

In [43]:
for i in range(60):
    print('>>>>>>>>>>>.',SGD_model_test.predict(X_test[i])) 
    print (y_test[i])

>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['webdesigner']
webdesigner
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['others']
others
>>>>>>>>>>>. ['webdesigner']
webde

In [44]:
svm_model_check = svm.SVC(kernel='linear') 
svm_model_check.fit(X_train, y_train) 
svm_prediction_check = svm_model_check.predict(X_test)
print ('svm_test:', np.mean(svm_prediction_check == y_test))

svm_test: 0.983333333333


In [45]:
SGD_model_check = SGDClassifier()
SGD_model_check.fit(X_train, y_train) 
SGD_prediction_check = SGD_model_check.predict(X_test)
print ('SGD_test:', np.mean(SGD_prediction_check == y_test))


SGD_test: 0.966666666667


/home/shabna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [46]:
from sklearn.ensemble import RandomForestClassifier  
RF_model_test = RandomForestClassifier(max_depth=2, random_state=0)
RF_model_test.fit(X_train, y_train) 
RF_prediction_test = RF_model_test.predict(X_test)
print ('RandomForest_test:', np.mean(RF_prediction_test == y_test))

from sklearn.tree import DecisionTreeClassifier
dtree_model_check = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train)
dtree_predictions_check = dtree_model_check.predict(X_test)
print ('DT_test:', np.mean(dtree_predictions_check == y_test))

from sklearn.neighbors import KNeighborsClassifier
knn_check = KNeighborsClassifier(n_neighbors = 7).fit(X_train, y_train)
knn_predictions_check = knn_check.predict(X_test) 
print ('knn_test:', np.mean(knn_predictions_check == y_test))

RandomForest_test: 0.966666666667
DT_test: 0.95
knn_test: 0.95


In [47]:
target_names=['Resume', 'JobDescription']

In [48]:

y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       1.00      0.97      0.98        29
JobDescription       0.97      1.00      0.98        31

   avg / total       0.98      0.98      0.98        60



In [49]:

y_pred = RF_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       0.97      0.97      0.97        29
JobDescription       0.97      0.97      0.97        31

   avg / total       0.97      0.97      0.97        60



In [50]:

y_pred = SGD_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       1.00      0.97      0.98        29
JobDescription       0.97      1.00      0.98        31

   avg / total       0.98      0.98      0.98        60



In [51]:

y_pred = dtree_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       1.00      0.90      0.95        29
JobDescription       0.91      1.00      0.95        31

   avg / total       0.95      0.95      0.95        60



In [52]:
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                precision    recall  f1-score   support

        Resume       1.00      0.90      0.95        29
JobDescription       0.91      1.00      0.95        31

   avg / total       0.95      0.95      0.95        60



In [53]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])
SGD_model_test.predict(test_file)[0]

'others'

In [54]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])
svm_model.predict(test_file)[0]

'others'

In [55]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])

RF_model.predict(test_file)[0]

'others'

In [56]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])
dtree_model.predict(test_file)[0]

'webdesigner'

In [57]:
test_file=vectorizer.transform(["/home/shabna/Desktop/example_codes/re/Resume/ios/48314531_India_2.09_yrs.pdf"])
knn.predict(test_file)[0]

'webdesigner'